In [1]:
import string
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.optimizers import SGD
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
import numpy as np
import os
from keras.models import load_model
from keras.models import model_from_json

ModuleNotFoundError: No module named 'keras'

In [ ]:
np.random.seed(123) 

In [ ]:
cd C:\Users\FDUSER.M-1737.000\dataset\bbc

In [ ]:
def mini_batch_generator(x, y, vocab, vocab_size, vocab_check, maxlen, batch_size=128):

    for i in range(0, len(x), batch_size):
        x_sample = x[i:i + batch_size]
        y_sample = y[i:i + batch_size]

        input_data = encode_data(x_sample, maxlen, vocab, vocab_size,
                                 vocab_check)

        yield (input_data, y_sample)

In [ ]:
def encode_data(x, maxlen, vocab, vocab_size, check):
    #Iterate over the loaded data and create a matrix of size maxlen x vocabsize
    #In this case that will be 1014x69. This is then placed in a 3D matrix of size
    #data_samples x maxlen x vocab_size. Each character is encoded into a one-hot
    #array. Chars not in the vocab are encoded into an all zero vector.

    input_data = np.zeros((len(x), maxlen, vocab_size))
    
    for dix, sent in enumerate(x):
        counter = 0
        sent_array = np.zeros((maxlen, vocab_size))
        chars = list(sent.lower().replace(' ', ''))
        for c in chars:
            if counter >= maxlen:
                pass
            else:
                char_array = np.zeros(vocab_size, dtype=np.int)
                if c in check:
                    ix = vocab[c]
                    char_array[ix] = 1
                sent_array[counter, :] = char_array
                counter += 1
        input_data[dix, :, :] = sent_array

    return input_data

In [ ]:
# This function creates a vocab of characters.

def create_vocab_set():
    #This alphabet is 69 chars vs. 70 reported in the paper since they include two
    # '-' characters. See https://github.com/zhangxiangxiao/Crepe#issues.

    alphabet = (list(string.ascii_lowercase) + list(string.digits) +
                list(string.punctuation) + ['\n'])
    
    vocab_size = len(alphabet)
    
    check = set(alphabet)

    vocab = {}
    reverse_vocab = {}
    
    for ix, t in enumerate(alphabet):
        vocab[t] = ix
        reverse_vocab[ix] = t

    return vocab, reverse_vocab, vocab_size, check

In [ ]:
def shuffle_matrix2(x, y):
    stacked = np.hstack((np.matrix(x).T, y))
    xi = np.array(stacked[:, 0]).flatten()
    yi = np.array(stacked[:, 1:])

    return xi, yi

In [ ]:
#Model params

#Filters for conv layers
nb_filter = 256

#Number of units in the dense layer
dense_outputs = 1024

#Conv layer kernel size
filter_kernels = [7, 7, 3, 3, 3, 3]

#Number of units in the final output layer. Number of classes.
cat_output = 5

In [ ]:
maindir = os.getcwd()
tempDirList = [x[0] for x in os.walk(maindir)]
dirList = [x[39:] for x in tempDirList if x[39:]]

In [ ]:
import glob
columns_list = ['Text', 'Class']

data_df = pd.DataFrame(columns=columns_list, index = None)

for dl in dirList:
    path = maindir + '\\' + dl # use your path
    allFiles = glob.glob(path + "\*.txt")
    for filename in allFiles:
        with open(filename, 'r') as file_:
            txt = file_.read()
            data_df = data_df.append({'Text': txt, 'Class':dl}, ignore_index=True) 

In [ ]:
data_df['News_Type'] = 0

In [ ]:
val = 0
for cls in dirList:
    data_df.loc[data_df['Class'] == cls, 'News_Type'] = val
    val = val + 1

In [ ]:
data_df = data_df.sample(frac=1).reset_index(drop=True)

In [ ]:
data_df = data_df.drop('Class', 1)

In [ ]:
data_df.shape

In [ ]:
from nltk.corpus import stopwords # Import the stop word list
import re
# Use regular expressions to do a find-and-replace

def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review) 
    #
    # 2. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 3. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 4. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 5. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [ ]:
# Get the number of reviews based on the dataframe column size
num_text = data_df["Text"].size

# Initialize an empty list to hold the clean reviews
clean_train_text = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range(0,num_text):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_text.append(review_to_words(data_df["Text"][i]))

In [ ]:
def load_ag_data2(data_df):

    # Get the number of reviews based on the dataframe column size
    num_text = data_df["Text"].size

    # Initialize an empty list to hold the clean reviews
    clean_train_text = []

    # Loop over each review; create an index i that goes from 0 to the length
    # of the movie review list 
    for i in range(0,num_text):
        # Call our function for each one, and add the result to the list of
        # clean reviews
        clean_train_text.append(review_to_words(data_df["Text"][i]))
        
    x_train = clean_train_text[:2000]
    x_train = np.array(x_train)
    
    y_train = data_df['News_Type']
    y_train = y_train[:2000]
    y_train = to_categorical(y_train)
    
    x_test = clean_train_text[2000:]
    x_test = np.array(x_test)
    
    y_test = data_df['News_Type']
    y_test = y_test[2000:]
    y_test = to_categorical(y_test)
    
    return (x_train, y_train), (x_test, y_test)

In [ ]:
(xt, yt), (x_test, y_test) = load_ag_data2(data_df)

In [ ]:
vocab, reverse_vocab, vocab_size, check = create_vocab_set()

In [ ]:
test_data = encode_data(x_test, maxlen, vocab, vocab_size, check)

In [ ]:
cd C:\\Users\\FDUSER.M-1737.000\\dataset\\AGNews

In [ ]:
model_name_path = 'params\\AG_model3.json'
model_weights_path = 'params\\AG_model_weights3.h5'

In [ ]:
json_file = open(model_name_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights(model_weights_path)

In [ ]:
model.summary()

In [ ]:
model.layers.pop()

In [ ]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

dropout_2 = model.layers[-1].output
#Output dense layer with softmax activation
pred = Dense(cat_output, activation='softmax', name='output')(dropout_2)

model = Model(input = model.input, output = pred)

model.compile(loss='categorical_crossentropy', optimizer = 'Adadelta' , metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
#Compile/fit params
batch_size = 80
nb_epoch = 10

In [ ]:
print('Fit model...')

for e in range(nb_epoch):
    
    xi, yi = shuffle_matrix(xt, yt)
    
    batches = mini_batch_generator(xi, yi, vocab, vocab_size, check, maxlen, batch_size=batch_size)

    accuracy = 0.0
    loss = 0.0
    step = 1
    print('Epoch: {}'.format(e))
    
    for x_train, y_train in batches:
        f = model.train_on_batch(x_train, y_train)
        loss += f[0]
        loss_avg = loss / step
        accuracy += f[1]
        accuracy_avg = accuracy / step
        if step % 10 == 0:
            print('  Step: {}'.format(step))
            print('\tLoss: {}. Accuracy: {}'.format(loss_avg, accuracy_avg))
        step += 1

print('\tLoss: {}. Accuracy: {}'.format(loss_avg, accuracy_avg))
print('Done Training')

In [ ]:
import json
model_name_path = 'params\\tl_1.json'
model_weights_path = 'params\\tl_1.h5'

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open(model_name_path, "w") as json_file:             
     json_file.write(model_json) 

# serialize weights to HDF5
model.save_weights(model_weights_path)
print("Saved model to disk")

In [ ]:
xi_test, yi_test = shuffle_matrix(x_test, y_test)    
test_batches = mini_batch_generator(xi_test, yi_test, vocab, vocab_size, check, maxlen, batch_size=batch_size)
test_accuracy = 0.0
test_loss = 0.0
test_step = 1
for x_test_batch, y_test_batch in test_batches:
        f_ev = model.test_on_batch(x_test_batch, y_test_batch)
        test_loss += f_ev[0]
        test_loss_avg = test_loss / test_step
        test_accuracy += f_ev[1]
        test_accuracy_avg = test_accuracy / test_step
        test_step += 1
    
print('Loss: {}. Accuracy: {}\n'.format(test_loss_avg, test_accuracy_avg))